In [1]:
# nuclio: start-code

In [ ]:
import sys, os
import ast
import json
import base64
import requests
import v3io.dataplane

MODULE_PATHS = ast.literal_eval(os.getenv('STEP_CONFIG'))['MODULE_PATHS']
module_paths = MODULE_PATHS or None
if module_paths != None:
    for module_path in module_paths:
        sys.path.append(module_path)
    
IMPORT_MODULES = ast.literal_eval(os.getenv('STEP_CONFIG'))['IMPORT_MODULES']
import_modules = IMPORT_MODULES or None
if import_modules != None:
    imports={}
    import importlib
    for  module in import_modules:
        imports[module] = importlib.import_module(module)

In [6]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [15]:
def output_stream(context,message,details):
    Records=[]
    # message must be a Python dictionary
    record = json.dumps(message).encode('utf-8')
    Records.append({
            "data":  record,
            "partition_key": message[context.user_data.unique_key],
            })
    try:
        response = context.user_data.v3io_client.put_records(container=context.user_data.step_config['OUTPUT_STREAM_CONTAINER'],
                                                 path=details['output_stream'],records=Records)
        if 'FailedRecordCount' in response:
            if response['FailedRecordCount'] > 0:
                context.logger.info(f"Failed to insert record {response} - {message}")
    except Exception as e:
        context.logger.info(f"Failed to insert record {e} - {message}")
        
    return

In [16]:
def igz_stream_init(context):
    setattr(context.user_data,'v3io_client',
            v3io.dataplane.Client(endpoint='http://v3io-webapi:8081', 
                                  access_key=os.getenv("V3IO_ACCESS_KEY")))

In [9]:
def output_function(context,message,details):
    _module, _function = details['post_process_function'].split(".")
    "imports['%s'].%s()"%(_module, _function)
    call_function = "imports['%s'].%s(context,message)"%(_module, _function)
    try:
        eval(call_function)
    except Exception as e: 
        context.logger.error(f"output_function failed: {e}")
    return

In [10]:
def output_http(context,message,details):
    context.logger.info("If I was working I would post to %s the key %s"% (details['url'],message[context.user_data.unique_key]))
    try:
        req = requests.post(url=details['url'],payload=json.loads(json.dumps(message)))
    except:
        context.logger.error(f"Failed to post message to {details['url']}")
        
    return

In [11]:
def post_process(context,message):
    for output in context.user_data.outputs:
        if 'condition' in output:
            try:
                if not eval(output['condition']):
                    continue
            except:
                context.logger.debug("Invalid condition spec %s"% output['condition'])
                continue
                
        call_function = "output_%s(context,message,output)"%output['kind']
        try:
            eval(call_function)
        except Exception as e: 
            context.logger.error(f"post_process failed: {e}")

In [12]:
def process(context,message):
    # message is dictionary loaded from the stream event
    _function="context.user_data.processing_class.%s(context,message)"%context.user_data.step_config['PROCESSING_FUNCTION']
    try:
        post_message = eval(_function)
    except Exception as e: 
        context.logger.error(f"process failed: {e}")
        return   
    post_process(context,post_message)
    
    return

In [13]:
def init_context(context):
    # IMPORTANTE: Load the model here
    # This code will put the model in the context of the function
    setattr(context.user_data,'step_config',ast.literal_eval(os.getenv('STEP_CONFIG')))
    _class_init = os.getenv('CLASS_INIT')
    _module, _class = context.user_data.step_config['CLASS_LOAD_FUNCTION'].split(".")
    if _class_init != None:
        _class_init = json.loads(_class_init)
        _load_class = "imports['%s'].%s(**_class_init)"%(_module, _class)
    else:
        _load_class = "imports['%s'].%s()"%(_module, _class)
        
    
    try:
        setattr(context.user_data,'processing_class',eval(_load_class))
    except Exception as e: 
        context.logger.error(f"Init class failed: {e}")
        
    setattr(context.user_data,'outputs',context.user_data.step_config['OUTPUTS'])
    _stream_output = False
    for output in context.user_data.outputs:
        if output['kind'] == 'stream':
            _stream_output = True
            
    if _stream_output:
        igz_stream_init(context)

    setattr(context.user_data,'unique_key',context.user_data.step_config['UNIQUE_KEY'])
        
    return
    

In [14]:
def handler(context, event):
    #context.logger.debug(event.body)
    try:
        message = json.loads(event.body.decode('utf-8'))
    except:
        print("Malformated Json ", event.body)
    process(context,message)
      
    return 

In [ ]:
# nuclio: end-code